In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import random
import torch.nn as nn
from dataset import Thyroid_Dataset
from model import Eff_Unet
from HarDMSEG import HarDMSEG
from loss_metric import DiceLoss, IOU_score, StructureLoss
from LightMed.model.LightMed import LightMed
from PMFSNet.lib.models.PMFSNet import PMFSNet
from PMFSNet.lib.models.PMFSNet_FFT import PMFSNet_FFT
from hybrid_model_v3 import HybridSegModel

/opt/anaconda3/envs/thyroid/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/thyroid/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/opt/anaconda3/envs/thyroid/lib/python3.12/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [2]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image
import os
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms.functional as tx
import random
import cv2
from PIL import ImageEnhance
from skimage.exposure import match_histograms
import torchvision.transforms as T
import torchvision.transforms.functional as F
class GE_Thyroid_Dataset(Dataset):
    def __init__(self, csv_file, transform, image_size):
        self.df = pd.read_csv(csv_file)
        self.transform = transform
        self.image_size = image_size
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_name = self.df["image_name"][idx]
        mask_name = self.df["mask_name"][idx]
        image_type = self.df["image_type"][idx]

        if image_type == "train":
            from_folder = "../ge_data/train"
        else:
            from_folder = "../ge_data/test"
        
        image_path = f"{from_folder}/images/{image_name}"
        mask_path = f"{from_folder}/masks/{mask_name}"

        image = Image.open(image_path).convert("L")
        mask = Image.open(mask_path).convert("L")
        
        image_tensor, mask_tensor = self.transform(image, mask, self.image_size)
        mask_tensor = (mask_tensor > 0.5).float()
        if torch.sum(mask_tensor) == 0:
            # print("nothing")
            return [None]
        return image_tensor, mask_tensor, mask_name, image_type

In [25]:
image_size = 128
batch_size = 1
def train_augmentation(image, mask, image_size):
    resize = T.Resize((image_size, image_size))
    image = resize(image)
    mask = resize(mask)

    p = 0.2
    
    # if(random.random() < p):
    #     jitter = T.ColorJitter(brightness = 0.5, contrast = 0.25)
    #     image = jitter(image)
    # if(random.random() < p):
    #     transform = T.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0))
    #     image = transform(image)  # image must be a PIL image
    # if(random.random() < p):
    #     angle = random.uniform(-10, 10)  # 旋轉角度從 ±10 度
    #     translate = (random.uniform(-0.05, 0.05) * image.size[0],
    #                  random.uniform(-0.05, 0.05) * image.size[1])  # 最多平移 ±5%
    #     scale = random.uniform(0.95, 1.05)  # 尺度縮放 ±5%
    #     shear = [random.uniform(-5, 5), random.uniform(-5, 5)]  # 小幅剪切
    
    #     image = F.affine(image, angle=angle, translate=translate, scale=scale, shear=shear)
    #     mask = F.affine(mask, angle=angle, translate=translate, scale=scale, shear=shear)

        
    image_tensor = tx.to_tensor(image)
    mask_tensor = tx.to_tensor(mask)

    # If standardization
    # mean = image_tensor.mean()
    # std = image_tensor.std()
    # std = std if std > 0 else 1.0  # avoid division by zero
    # image_tensor = (image_tensor - mean) / std
    return image_tensor, mask_tensor
def test_augmentation(image, mask, image_size):
    resize = T.Resize((image_size, image_size))
    image = resize(image)
    mask = resize(mask)

    image_tensor = tx.to_tensor(image)
    mask_tensor = tx.to_tensor(mask)

    # If standardization
    # mean = image_tensor.mean()
    # std = image_tensor.std()
    # std = std if std > 0 else 1.0  # avoid division by zero
    # image_tensor = (image_tensor - mean) / std
    return image_tensor, mask_tensor
def custom_collate_fn(batch):
    # print(batch)
    filtered_batch = [item for item in batch if item[0] is not None]
    if len(filtered_batch) == 0:
        return [None, None, None, None]
    return torch.utils.data.dataloader.default_collate(filtered_batch)
train_dataset = GE_Thyroid_Dataset("../ge_data/train_csv_for_finetune(1).csv", transform = train_augmentation, image_size = image_size)
# train_dataset = GE_Thyroid_Dataset("../ge_data/test_csv_for_finetune_all.csv", transform = train_augmentation, image_size = image_size)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True, collate_fn=custom_collate_fn)

test_dataset = GE_Thyroid_Dataset("../ge_data/test_csv_for_finetune_all.csv", transform = test_augmentation, image_size = image_size)
# test_dataset = GE_Thyroid_Dataset("../ge_data/train_csv_for_finetune(0.3).csv", transform = test_augmentation, image_size = image_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False, collate_fn=custom_collate_fn)



In [26]:
image.shape

torch.Size([1, 1, 128, 128])

In [27]:
print(len(train_dataset), len(test_dataset))

565 112


In [35]:
def get_pos(mask):
    row = []
    col = []
    for i in range(mask.shape[0]):
        for j in range(mask.shape[1]):
            if mask[i][j]:
                row.append(i)
                col.append(j)
    return np.mean(row), np.mean(col)

In [36]:
train_mean = []
train_std = []
train_size_ratio = []
train_row = []
train_col = []
for image_tensor, mask_tensor, mask_name, image_type in tqdm(train_dataloader):
    # print(image_tensor)
    if image_tensor == None:
        continue
    mean = torch.mean(image_tensor[0][0])
    std = torch.std(image_tensor[0][0])
    size_ratio = torch.sum(mask_tensor[0][0]) / (mask_tensor[0][0].shape[0] * mask_tensor[0][0].shape[1])
    row, col = get_pos(mask_tensor[0][0])
    train_row.append(row)
    train_col.append(col)
    train_mean.append(mean)
    train_std.append(std)
    train_size_ratio.append(size_ratio)
print("train mean : ", np.mean(train_mean))
print("train std : ", np.mean(train_std))
print("train size ratio : ", np.mean(train_size_ratio))
print("train row : ", np.mean(train_row))
print("train col : ", np.mean(train_col))

100%|█████████████████████████████████████████| 565/565 [00:11<00:00, 49.38it/s]

train mean :  0.17272177
train std :  0.19266826
train size ratio :  0.05343415
train row :  35.85483061201079
train col :  62.27568023225343


In [37]:
test_mean = []
test_std = []
test_size_ratio = []
test_row = []
test_col = []
for image_tensor, mask_tensor, mask_name, image_type in tqdm(test_dataloader):
    # print(image_tensor)
    if image_tensor == None:
        continue
    mean = torch.mean(image_tensor[0][0])
    std = torch.std(image_tensor[0][0])
    size_ratio = torch.sum(mask_tensor[0][0]) / (mask_tensor[0][0].shape[0] * mask_tensor[0][0].shape[1])
    row, col = get_pos(mask_tensor[0][0])
    test_row.append(row)
    test_col.append(col)
    test_mean.append(mean)
    test_std.append(std)
    test_size_ratio.append(size_ratio)
print("test mean : ", np.mean(test_mean))
print("test std : ", np.mean(test_std))
print("test size ratio : ", np.mean(test_size_ratio))
print("test row : ", np.mean(test_row))
print("test col : ", np.mean(test_col))

100%|█████████████████████████████████████████| 112/112 [00:02<00:00, 48.70it/s]

test mean :  0.1537063
test std :  0.15427291
test size ratio :  0.0825576
test row :  46.099830831116854
test col :  64.7797866359704
